# Performance assessment

## 1. Collect the measurements

First, we need to collect as clean measurements as possible.

Prepare your system:
- Disable turboboost (check `intel_pstate` is the active driver with `cat /sys/devices/system/cpu/cpu*/cpufreq/scaling_driver`; then execute `echo "1" | sudo tee /sys/devices/system/cpu/intel_pstate/no_turbo`; revert after measurement)
- Fix the frequency of some core using `cpupower frequency-set -d 1900Mhz`
- Set `echo 1000000 > /proc/sys/kernel/sched_rt_runtime_us` to enable real-time process to use 100% of CPU (no scheduling)
- Start the process with `taskset --cpu-list 2 chrt -f 99 php tests/performance/bin.php` (run on core 2 as a real-time process)

## 2. Load & cleanup data

First, we cleanup the data to remove outliers.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import math as mathlib

In [ ]:
def load_json_from_file(file_path):
  with open(file_path, 'r') as f:
    return json.load(f)

iterations = 1000
measurements = []
for file in os.listdir('measurements'):
    if file.endswith("_" + str(iterations) + ".json"):
        full_path = os.path.join('measurements', file)
        content = load_json_from_file(full_path)
        measurements.append(content)

### 2.1 Filter outliers

Each measurement should have more or less the same timing.

On a real machine, reaching this is hard, even when configuring it properly (see chapter 1). Hence we remove outlier measurements. You can adjust the aggressiveness with the following parameter: `selectivity` defines how many iterations are kept, i.e. only one out of `selectivity` measurements.

In [ ]:
def clean_measurement(measurement, selectivity):
    sorted_measurements = sorted(measurement['measurements'])
    selectivity_index = int(len(sorted_measurements) / (selectivity * 2))
    lower_bound = sorted_measurements[selectivity_index]
    upper_bound = sorted_measurements[selectivity_index*3]

    cleaned_measurements = [m for m in measurement['measurements'] if lower_bound <= m <= upper_bound]

    cleaned_measurement = measurement.copy()
    cleaned_measurement['measurements'] = cleaned_measurements

    return cleaned_measurement

selectivity = 2

clean_measurements = []
for measurement in measurements:
    cleaned_measurement = clean_measurement(measurement, selectivity)
    clean_measurements.append(cleaned_measurement)

**Check the graph.** If the outlier filtering is effective, then the timings should all be very similar. Concretely, you should see a graphs that are all essentially just noise (lots of blue, almost filling out the graph).

In [ ]:
columns = 3
rows = mathlib.ceil(len(clean_measurements) / columns)
fig, axes = plt.subplots(rows, columns, figsize=(14, 5*rows))

for row in range(0, rows):
    for column in range(0, columns):
        axe = axes[row][column]
        measurement_index = row * columns + column
        if measurement_index >= len(clean_measurements):
            fig.delaxes(axe)
            continue
        measurement = clean_measurements[measurement_index]

        axe.plot(measurement["measurements"], label=f'measurements of {measurement["math"]} over {measurement["curve"]}')
        axe.set_xlabel('measurement')
        axe.set_ylabel('time')
        axe.legend()

plt.show()

### 3. Assess performance

If chapter 2 does not show signs of a botched measurement, we can now proceed to assess the measurement result.

In [ ]:
averages = {}
for measurement in clean_measurements:
    average = __builtins__.sum(measurement['measurements']) / len(measurements)

    curve = measurement['curve']
    if curve not in averages:
        averages[curve] = {}

    math = measurement['math']
    if math not in averages[curve]:
        averages[curve][math] = {}

    averages[curve][math] = average

### 3.1. Graphical

We output the performance in graphs per curve type.

In [ ]:
math_combinations = []
for curve in averages:
    graph_template = averages[curve].keys()

    exists = False
    for existing_math_combination in math_combinations:
        if set(existing_math_combination) == set(graph_template):
            exists = True
            break

    if not exists:
        unsafe_math = sorted([x for x in graph_template if x.endswith('UnsafeMath')])
        other_math = sorted([x for x in graph_template if not x.endswith('UnsafeMath')])

        math_combinations.append(unsafe_math + other_math)

In [ ]:
columns = 2
rows = mathlib.ceil(len(math_combinations) / columns)
fig, axes = plt.subplots(rows, columns, figsize=(14, 5*rows))

for row in range(0, rows):
    for column in range(0, columns):
        axe = axes[row][column]
        math_combination_index = row * columns + column
        if math_combination_index >= len(math_combinations):
            fig.delaxes(axe)
            continue
        math_combination = math_combinations[math_combination_index]

        relevant_averages = {}
        for curve in sorted(averages.keys()):
            if set(averages[curve].keys()) != set(math_combination):
                continue

            relevant_averages[curve] = averages[curve]

        math_averages = {}
        for math in math_combination:
            math_averages[math] = []

        for curve in sorted(relevant_averages.keys()):
            for math in relevant_averages[curve]:
                math_averages[math].append(relevant_averages[curve][math])

        w, x = 1/(len(math_averages)+1), np.arange(len(relevant_averages))
        for index, math in enumerate(math_averages):
            axe.bar(x + w*index - (len(math_averages)-1)*w/2, math_averages[math], width=w, label=f'{math}')
        axe.set_ylabel('time')
        axe.set_xticks(x)
        axe.set_xticklabels(sorted(relevant_averages.keys()))
        axe.legend()

        axe.tick_params(axis='x', rotation=20)

plt.show()

## 3.2 Tabular

Visualized as table

In [ ]:
for math_combination in math_combinations:
    header = '|   |'
    subheader = '|---|'
    for math in math_combination:
        header += "`" + str(math) + "` |"
        subheader += '---|'
    print(header)
    print(subheader)

    for curve in sorted(averages.keys()):
        if set(averages[curve].keys()) != set(math_combination):
            continue

        row = '|`' + curve + "`|"
        for math in math_combination:
            row += str(round(averages[curve][math], 3)) + "|"
        print(row)

    print("\n\n")